In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join('..', 'src')))
sys.setrecursionlimit(20500)
import pandas as pd
import spacy

import matplotlib.pyplot as plt
import clean_dataset as clean
import make_dataset as mk
import vectorize_embed as em
import tools as tools
import models as m

import pickle
import numpy as np

'''features'''
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import label_binarize

'''Classifiers'''
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.pipeline import Pipeline

'''Metrics/Evaluation'''
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
from scipy import interp
from itertools import cycle
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline

import joblib
import seaborn as sns

[nltk_data] Downloading package stopwords to C:\Users\Jonas
[nltk_data]     Nothnagel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Jonas
[nltk_data]     Nothnagel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Jonas
[nltk_data]     Nothnagel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Jonas
[nltk_data]     Nothnagel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
'''import data'''
df = pd.read_csv(os.path.abspath(os.path.join('..', 'data/processed/'))+'/taxonomy_final.csv')  
df['all_text_clean_spacy'] = df['all_text_clean_spacy'].astype(str).apply(clean.basic)
df['all_text_clean'] = df['all_text_clean'].astype(str)

In [3]:
cats = df.drop(columns=['PIMS_ID','all_text_clean', 'all_text_clean_spacy'])
categories = cats.columns.tolist()

In [5]:
# iterate over categories and save best models:
    #Creating a dict of the models
model_dict = {
              'LR_balanced' : LogisticRegression(random_state = 3, class_weight = "balanced"),
              'LR' : LogisticRegression(random_state = 3),
              'SDG' : SGDClassifier(random_state=3, loss='log'),
              'SDG_balanceed' : SGDClassifier(random_state=3, loss='log',class_weight = "balanced" ),
              'RF': RandomForestClassifier(random_state=3),
              'Decsision_Tree': DecisionTreeClassifier(random_state=3),
              'AdaBoost': AdaBoostClassifier(random_state=3),
              'GNB': GaussianNB(),
              'KNB': KNeighborsClassifier()}
    
for category in categories: 
    if category != "no tag":
        if df[category].sum(axis=0) > 20:
            
            print('processing:', category)
            print('_____')
            #Creating the features (tf-idf weights) for the processed text
            texts = df['all_text_clean_spacy'].astype('str')

            tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), 
                                               min_df = 2, 
                                               max_df = .95)
            X = tfidf_vectorizer.fit_transform(texts)     
            y = df[category].values

            if len(category) > 50:
                 shorter = category[0:20]
                 #save vectorizer:
                 filename = '../models/tf_idf/tf_idf_lsa/'+shorter+'_'+'vectorizer.sav'
                 joblib.dump(tfidf_vectorizer, filename)    

            else:
                #save vectorizer:
                filename = '../models/tf_idf/tf_idf_lsa/'+category+'_'+'vectorizer.sav'
                joblib.dump(tfidf_vectorizer, filename)    

            #Dimenionality reduction. Only using the 100 best features er category
            lsa = TruncatedSVD(n_components=100, 
                               n_iter=10, 
                               random_state=3)
            X = lsa.fit_transform(X)

            if len(category) > 50:
                print('long')
                shorter = category[0:20]
                #save lsa model:
                filename = '../models/tf_idf/tf_idf_lsa/'+shorter+'_'+'lsa.sav'
                joblib.dump(lsa, filename)
            else:
                #save vectorizer:
                filename = '../models/tf_idf/tf_idf_lsa/'+category+'_'+'lsa.sav'
                joblib.dump(lsa, filename)  

            #Train test split with stratified sampling for evaluation
            X_train, X_test, y_train, y_test = train_test_split(X, 
                                                                y, 
                                                                test_size = .3, 
                                                                shuffle = True, 
                                                                stratify = y, 
                                                                random_state = 3)

            m.model_score_df_all(model_dict, category, 'tf_idf_lsa', X_train, X_test, y_train, y_test)

processing: forests
_____
processing: conserve_areas
_____
processing: freshwaters
_____
processing: grasslands
_____
processing: marine
_____
processing: wetlands
_____
processing: human_altered_areas
_____
processing: coasts
_____
processing: coral_reefs
_____
processing: dryland_forests
_____
processing: indigenous_and_communities_conserved_areas_iccas
_____
processing: industrial_site
_____
processing: key_biodiversity_areas_kbas
_____
processing: marine_and_coastal_protected_areas
_____
processing: montane_forests
_____
processing: productive_landscapes_seascapes
_____
processing: rivers_and_river_basins
_____
processing: rural_areas
_____
processing: terrestrial_protected_areas
_____
processing: tropical_forests
_____
processing: urban_areas
_____
processing: large_marine_ecosystem
_____
processing: chemicals_and_waste_programme
_____
processing: climate_change_adaptation_programme
_____
processing: climate_forest_of_climate_change_mitigation
_____
processing: climate_strategies_